<a href="https://colab.research.google.com/github/damashup/angular-socket-io-im/blob/master/CAS_Script_1_Sorare_Accounting_MyTransactions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
# Cell 0: CONFIGURATION

# Set to True to run the heavy raw data dumps for troubleshooting.
# Set to False for faster standard runs (this skips RAW_API_Dump, STANDARDIZED_RAW_Dump, and RAW_CRAFTS_AUDIT).
RUN_RAW_DUMPS = False

# SCRIPT 1: SNAPSHOT HELPER FUNCTIONS (REVISED TO USE DIRECT DRIVE API)

# Globals for Drive service
DRIVE_SERVICE = None
SNAPSHOT_FOLDER_NAME = "SorareReportsSnapshots"

def initialize_drive_service():
    """Initializes the Google Drive API service using gspread's credentials."""
    global DRIVE_SERVICE
    if DRIVE_SERVICE is None:
        try:
            # Re-authorize to ensure Drive scope is captured
            creds, _ = default()
            DRIVE_SERVICE = build('drive', 'v3', credentials=creds)
            print("Drive API service initialized.")
        except Exception as e:
            print(f"Failed to build Drive Service: {e}")
            return False
    return True

def get_snapshot_folder_id(folder_name):
    """Finds the ID of the snapshot folder, creating it if necessary."""
    if not initialize_drive_service():
        return None

    # Search for the folder by name in the root folder
    q = f"name='{folder_name}' and mimeType='application/vnd.google-apps.folder' and 'root' in parents and trashed=false"

    response = DRIVE_SERVICE.files().list(q=q, spaces='drive', fields='files(id)').execute()
    files = response.get('files', [])

    if files:
        return files[0]['id']
    else:
        # Create the folder
        file_metadata = {
            'name': folder_name,
            'mimeType': 'application/vnd.google-apps.folder',
            'parents': ['root']
        }
        new_folder = DRIVE_SERVICE.files().create(body=file_metadata, fields='id').execute()
        print(f"Created new Drive folder: '{folder_name}'.")
        return new_folder['id']

def create_and_store_snapshot(source_file_id, source_file_name, target_folder_id):
    """Duplicates the source spreadsheet and moves the copy to the target folder."""
    if not initialize_drive_service() or not target_folder_id:
        print("Snapshot failed: Drive service not ready or target folder missing.")
        return

    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    snapshot_title = f"{source_file_name}_Snapshot_{timestamp}"

    try:
        # 1. Copy the file
        copied_file_metadata = {'name': snapshot_title}
        copied_file = DRIVE_SERVICE.files().copy(
            fileId=source_file_id,
            body=copied_file_metadata,
            fields='id, parents'
        ).execute()
        copied_file_id = copied_file['id']

        # 2. Move the copied file to the target folder
        # Remove original parent ('root') and add new parent (snapshot folder)

        # Get existing parents (usually just 'root')
        current_parents = ','.join(copied_file.get('parents'))

        DRIVE_SERVICE.files().update(
            fileId=copied_file_id,
            addParents=target_folder_id,
            removeParents=current_parents,
            fields='id, parents'
        ).execute()

        print(f"✅ Snapshot '{snapshot_title}' successfully created and stored in '{SNAPSHOT_FOLDER_NAME}'.")

    except Exception as e:
        print(f"❌ Snapshot creation failed: {e}")

In [49]:
# CELL 1 (UPDATED IMPORTS)

# Import the required libraries
import requests
import csv
import datetime
import time
import math
import os
import pandas as pd # Needed for Drive helper functions
from google.colab import drive
from google.colab import auth
from google.auth import default # Needed for Drive service auth
from googleapiclient.discovery import build # CRITICAL: This defines the 'build' function!

# --- END IMPORTS ---


# PREREQUISITE
def is_drive_mounted(path='/content/drive'):
  return os.path.exists(path)

if not is_drive_mounted():
  drive.mount('/content/drive')

In [50]:
# Cell 2 - PREREQUISITE - SORARE

from google.colab import userdata

# Retrieve the key securely from Colab Secrets
SORARE_API_KEY = userdata.get('SORARE_API_KEY')
SORARE_JWT = userdata.get('SORARE_JWT')

In [51]:
# Cell 3 VARIBLES

user = "nanzo"

# Define the GraphQL endpoint URL
url = "https://api.sorare.com/federation/graphql"

# Define Sorare API key:
api_key = SORARE_API_KEY
jwt = SORARE_JWT

# Define the path to the CSV file in your Google Drive
output_csv_path = "/content/drive/MyDrive/SorareDumps/SA_MyTransactions.csv"

# Define the headers for the CSV files
headers = [
    "transaction_type",
    "card_slug",                # Card bought OR Card sold
    "card_url",                 # URL for the card slug
    "sender_slug",
    "actual_receiver_slug",
    "token_op_type",            # <-- NEW COLUMN: The raw GraphQL typename
    "reward_id",                # <-- NEW COLUMN for Reward Tracking
    "reward_slug",              # <-- NEW COLUMN for SO5 Reward Slug
    "fiat_value_gbp",           # Buy Price (Negative) OR Sell Price (Positive)
    "crypto_value_eth",         # Buy Price (Negative) OR Sell Price (Positive)
    "reference_currency",
    "cards_exchanged_slugs",    # Cards received in a trade OR Cards sent in a trade
    "transaction_date",
    "transaction_time",
    "status",
    "pl_value_gbp",
    "current_time",
    "account_entry_id",
    "token_operation_id" # <--- NEW COLUMN ADDEDD
]

# Define your Google Sheet names
GOOGLE_SHEET_NAME = "Sorare Transactions Dump"
SALES_WORKSHEET_NAME = "AccountTransactions_Sales"
BUYS_WORKSHEET_NAME = "AccountTransactions_Buys"
REWARDS_WORKSHEET_NAME = "AccountTransactions_MonetaryRewards"
RAW_WORKSHEET_NAME = "RAW_API_Dump"
STANDARDIZED_RAW_WORKSHEET_NAME = "STANDARDIZED_RAW_Dump"
ANOMALIES_WORKSHEET_NAME = "Anomalies_Excluded_Data"
CARD_REWARDS_WORKSHEET_NAME = "MyRewards_Cards"
CRAFTED_CARDS_WORKSHEET_NAME = "MyCrafts_Cards"
RAW_CRAFTS_AUDIT_WORKSHEET_NAME = "RAW_CRAFTS_AUDIT"
NOTIONAL_SALES_WORKSHEET_NAME = "Notional_Sales_Auto" # <-- RENAMED SHEET
DIRECT_OFFERS_MANUAL_WORKSHEET_NAME = "Direct_Offers_Manual_Audit"
MANUAL_SALES_WORKSHEET_NAME = "Notional_Sales_Manual" # NEW SHEET NAME


In [52]:
# Cell 4: API QUERIES

# Define the GraphQL query
query = """
query GetAllTransactionsForUser($cursor: String) {
  currentUser {
    slug
    accountEntries(first: 100, after: $cursor) {
      nodes {
        id
        aasmState
        amounts {
          gbpCents
          referenceCurrency
          wei
        }
        date
        provisional
        tokenOperation {
          ... on TokenOffer {
            id
            actualReceiver {
              ... on User {
                slug
              }
            }
            marketFeeAmounts {
              gbpCents
            }
            receiver {
              ... on User {
                slug
              }
            }
            receiverSide {
              amounts {
                gbpCents
                referenceCurrency
                wei
              }
              anyCards {
                slug
              }
            }
            sender {
              ... on User {
                slug
              }
            }
            senderSide {
              amounts {
                gbpCents
                referenceCurrency
                wei
              }
              anyCards {
                slug
              }
            }
          }
          ... on TokenBid {
            id
            userBidder {
              slug
            }
            auction {
              anyCards {
                slug
              }
            }
          }
          ... on TokenPrimaryOffer {
            id
            buyer {
              slug
            }
            anyCards {
              slug
            }
          }
          ... on TokenMonetaryReward {
            id
            rewardId
            sport
          }
          ... on So5Reward {
            id
            slug
            rewardCards {
              anyCard {
                slug
              }
            }
          }
          __typename
        }
      }
      pageInfo {
        endCursor
        hasNextPage
      }
    }
  }
}
"""

In [53]:
# Cell 5 REWARDS API QUERY

# Define the rewards GraphQL query
rewards_query = """
query MyRewards($slug: String!, $cursor: String) {
  user(slug: $slug) {
    rewardedRankings(first: 100, after: $cursor) {
      nodes {
        so5Rewards {
          id
          slug
          so5Leaderboard {
            gameWeek
            endDate
          }
          amount {
            gbpCents
            referenceCurrency
            wei
          }
          rewardCards {
            anyCard {
              slug
            }
          }
					rewards {
            id
            ... on AnyCardReward {
              id
              aasmState
            }
            ... on MonetaryReward {
              id
              aasmState
            }
            __typename
          }
        }
      }
      pageInfo {
        endCursor
        hasNextPage
      }
    }
  }
}
"""

# Define the crafts GraphQL query (CRITICAL ADDITION)
crafts_query = """
query MyCrafts($cursor: String, $rarity: Rarity!) {
  currentUser {
    slug
    cardShardsHistoryTransactions(
      first: 100
      after: $cursor
      rarity: $rarity
      sport: FOOTBALL
    ) {
      nodes {
        date
        description
        id
        label
      }
      pageInfo {
        endCursor
        hasNextPage
      }
    }
  }
}
"""

# --- NEW: Direct Offers GraphQL Query ---
direct_offers_query = """
query GetAllDirectOffersForUser($direction: OfferDirection!, $cursor: String, $first: Int) {
    currentUser {
      slug
      tokenOffers(direction: $direction, first: $first, after: $cursor,states:ACCEPTED) {
        totalCount
        nodes {
          id
          status
          acceptedAt
          receiver {
            ... on User {
              slug
            }
          }
          receiverSide {
            amounts {
              gbpCents
              wei
              referenceCurrency
            }
            anyCards {
              slug
            }
          }
          sender {
            ... on User {
              slug
            }
          }
          senderSide {
            amounts {
              gbpCents
              wei
              referenceCurrency
            }
            anyCards {
              slug
            }
          }
        }
        pageInfo {
          endCursor
          hasNextPage
        }
      }
    }
  }
"""

In [54]:
# Cell 6: REWARDS API FETCH LOOP (Updated to handle new query structure)

def fetch_all_rewards(query, user_slug, api_key, jwt):
    all_rewards = []
    variables = {"slug": user_slug, "cursor": None}
    url = "https://api.sorare.com/federation/graphql" # Use global URL
    has_next_page = True

    # Set up the request headers (copied from execute_query)
    headers = {
        "Content-Type": "application/json",
        "APIKEY": api_key,
        "JWT-AUD": "MyNanzo",
        "Authorization": "Bearer " + jwt
    }

    print("Fetching rewards data...")

    while has_next_page:
        payload = {"query": query, "variables": variables}
        response = requests.post(url, headers=headers, json=payload)
        print(f"Status Code: {response.status_code}")

        if response.status_code != 200:
            print(f"Stopping rewards fetch due to HTTP error {response.status_code}.")
            break

        response_json = response.json()

        if 'errors' in response_json:
            print(f"GraphQL Errors: {response_json['errors']}")
            break

        # Navigate to the correct nested nodes array
        try:
            # The structure is: data -> user -> rewardedRankings
            rewarded_rankings = response_json["data"]["user"]["rewardedRankings"]
            current_nodes = rewarded_rankings["nodes"]
            page_info = rewarded_rankings["pageInfo"]
        except KeyError as e:
            # Capture the full JSON output for inspection
            print(f"Error: Could not find key '{e}' in response. Stopping.")
            # We already have the JSON in the response_json variable from above
            print(f"Full JSON: {response_json}")
            break

        all_rewards.extend(current_nodes)

        variables["cursor"] = page_info["endCursor"]
        has_next_page = page_info["hasNextPage"]

    print(f"Total rewards entries fetched: {len(all_rewards)}")
    return all_rewards

In [55]:
# Cell 7: CRAFTS API FETCH LOOP (Case Sensitivity Fix)

def fetch_all_crafts(query, api_key, jwt):
    all_crafts = []
    # Rarity names must be lowercase as expected by the API
    rarities = ["unique", "super_rare", "rare", "limited"]
    url = "https://api.sorare.com/federation/graphql"

    # Set up headers once
    headers = {
        "Content-Type": "application/json",
        "APIKEY": api_key,
        "JWT-AUD": "MyNanzo",
        "Authorization": "Bearer " + jwt
    }

    print("\nFetching crafted card data for all rarities...")

    for rarity in rarities:
        current_crafts = []
        # FIX: Send rarity in lowercase (e.g., "unique" instead of "UNIQUE")
        variables = {"cursor": None, "rarity": rarity}
        has_next_page = True

        print(f"  -> Fetching {rarity} crafts...")

        while has_next_page:
            payload = {"query": query, "variables": variables}
            response = requests.post(url, headers=headers, json=payload)

            if response.status_code != 200:
                print(f"    Stopping fetch due to HTTP error {response.status_code}.")
                break

            response_json = response.json()

            if 'errors' in response_json:
                print(f"    GraphQL Errors: {response_json['errors']}. Stopping.")
                break

            try:
                # Structure: data -> currentUser -> cardShardsHistoryTransactions
                craft_data = response_json["data"]["currentUser"]["cardShardsHistoryTransactions"]
                current_nodes = craft_data["nodes"]
                page_info = craft_data["pageInfo"]
            except KeyError:
                print("    Error: Craft data structure is unexpected. Stopping.")
                break

            current_crafts.extend(current_nodes)

            variables["cursor"] = page_info["endCursor"]
            has_next_page = page_info["hasNextPage"]

        all_crafts.extend(current_crafts)

    print(f"Total crafted transactions fetched: {len(all_crafts)}")
    return all_crafts

In [56]:
# Cell 8: DIRECT OFFERS API FETCH LOOP

def fetch_all_direct_offers(query, user_slug, api_key, jwt):
    """
    Fetches all SENT and RECEIVED direct offers and returns only those with 'accepted' status.
    """
    all_accepted_offers = []
    directions = ["SENT", "RECEIVED"] # Fetch both directions
    url = "https://api.sorare.com/federation/graphql"

    headers = {
        "Content-Type": "application/json",
        "APIKEY": api_key,
        "JWT-AUD": "MyNanzo",
        "Authorization": "Bearer " + jwt
    }

    print("\nFetching ACCEPTED direct offer data for SENT and RECEIVED directions...")

    for direction in directions:
        # Note: The 'states:ACCEPTED' filter is now in the query itself.
        # We need to set 'first' parameter explicitly as per the query structure.
        variables = {"direction": direction, "cursor": None, "first": 100}
        has_next_page = True
        current_offers_count = 0

        print(f"  -> Fetching {direction} offers...")

        while has_next_page:
            payload = {"query": query, "variables": variables}
            response = requests.post(url, headers=headers, json=payload)

            if response.status_code != 200:
                print(f"    Stopping fetch due to HTTP error {response.status_code}.")
                break

            response_json = response.json()

            if 'errors' in response_json:
                print(f"    GraphQL Errors: {response_json['errors']}. Stopping.")
                break

            try:
                offer_data = response_json["data"]["currentUser"]["tokenOffers"]
                current_nodes = offer_data["nodes"]
                page_info = offer_data["pageInfo"]
            except KeyError:
                print("    Error: Direct offer data structure is unexpected. Stopping.")
                break

            # Store all nodes, as they are pre-filtered by the API query
            for offer in current_nodes:
                offer['direction_role'] = direction
                all_accepted_offers.append(offer)
                current_offers_count += 1

            variables["cursor"] = page_info["endCursor"]
            has_next_page = page_info["hasNextPage"]

        print(f"  -> Found {current_offers_count} accepted {direction} offers.")


    print(f"Total accepted direct offers fetched: {len(all_accepted_offers)}")
    return all_accepted_offers

In [57]:
# CEll 9 FUNCTIONS

#SORARE API CALL
# Define a function to execute the query
def execute_query(query, variables, api_key, jwt):
    # Set up the request headers and payload
    headers = {
        "Content-Type": "application/json",
        "APIKEY": api_key,
        "JWT-AUD": "MyNanzo",
        "Authorization": "Bearer " + jwt
    }
    payload = {"query": query, "variables": variables}

    # Send the request to the API endpoint
    response = requests.post(url, headers=headers, json=payload)
    print(f"Status Code: {response.status_code}")

    response_json = response.json()

    # Check for GraphQL errors
    if 'errors' in response_json:
        print(f"GraphQL Errors: {response_json['errors']}")
        return None

    # Parse the response JSON
    try:
        # Navigate to the 'accountEntries' object directly
        account_entries = response_json["data"]["currentUser"]["accountEntries"]
    except KeyError as e:
        # Print the full response for better debugging if the expected structure is missing
        print(f"Error: Could not find key in response. Full JSON: {response_json}")
        raise e

    return account_entries # account_entries is {'nodes': [...], 'pageInfo': {...}}


# Helper function to safely get a nested value
def get_nested(data, keys, default=None):
   # Safely retrieves a nested value from a dictionary
   if data is None:
       return default
   value = data
   for key in keys:
       if isinstance(value, dict) and key in value:
           value = value[key]
       elif isinstance(value, list) and isinstance(key, int) and key < len(value):
           value = value[key]
       else:
           return default
   return value


# Function to convert Wei (as a string) to ETH (as a float)
def wei_to_eth(wei_str):
   """Converts a wei string value to a floating point ETH value."""
   try:
       # Convert the string to an integer
       wei = int(wei_str)
       # 1 ETH = 10^18 Wei
       eth = wei / 10**18
       # Round to 8 decimal places for reasonable precision
       return round(eth, 8)
   except (ValueError, TypeError):
       return 0.0

# -------------------------------------------------------------
# --- WRITING FUNCTIONS (Updated Overwrite Logic) ---
# -------------------------------------------------------------

def write_raw_data(transactions_list, sheet_name, raw_worksheet_name):
    import json
    raw_headers = ["ID", "Raw JSON Data"]

    try:
        sh = gc.open(sheet_name)
    except gspread.SpreadsheetNotFound:
        print(f"Spreadsheet '{sheet_name}' not found. Skipping raw data dump.")
        return

    try:
        wks_raw = sh.worksheet(raw_worksheet_name)
        wks_raw.clear() # <--- FIX: Clear existing content if sheet is found
    except gspread.WorksheetNotFound:
        wks_raw = sh.add_worksheet(title=raw_worksheet_name, rows="100", cols="2")

    # Prepare data
    rows_to_write = []
    for i, tx in enumerate(transactions_list):
        transaction_id = tx.get('id', f'ROW_{i+1}')
        raw_json_string = json.dumps(tx)
        rows_to_write.append([transaction_id, raw_json_string])

    data_to_overwrite = [raw_headers] + rows_to_write

    if data_to_overwrite:
        num_rows = len(data_to_overwrite)
        num_cols = len(raw_headers)
        range_label = f'A1:{gspread.utils.rowcol_to_a1(num_rows, num_cols)}'

        wks_raw.update(range_name=range_label, values=data_to_overwrite, value_input_option='RAW')

        print(f"Successfully OVERWROTE {len(rows_to_write)} rows of RAW JSON data to '{raw_worksheet_name}'.")
    else:
        # Clear and write headers if no data
        wks_raw.clear()
        wks_raw.append_row(raw_headers)
        print("No raw data to dump to Google Sheet. Headers were refreshed.")


def write_standardized_raw_data(transactions_list, sheet_name, standardized_worksheet_name):
    standardized_headers = [
        "ID", "AASM_State", "Date_Time", "GBP_Cents", "Ref_Currency", "WEI_Amount",
        "TokenOp_Type", "TokenOp_Sender", "TokenOp_Receiver", "TokenOp_ActualReceiver",
        "SenderSide_Cards", "ReceiverSide_Cards", "Auction_Card_Slug"
    ]

    try:
        sh = gc.open(sheet_name)
    except gspread.SpreadsheetNotFound:
        print(f"Spreadsheet '{sheet_name}' not found. Skipping standardized raw data dump.")
        return

    try:
        wks_std = sh.worksheet(standardized_worksheet_name)
        wks_std.clear() # <--- FIX: Clear existing content if sheet is found
    except gspread.WorksheetNotFound:
        wks_std = sh.add_worksheet(title=standardized_worksheet_name, rows="100", cols=str(len(standardized_headers)))


    rows_to_write = []
    for i, tx in enumerate(transactions_list):
        token_op = tx.get('tokenOperation', {}) if tx.get('tokenOperation') is not None else {}
        # ... (populating rows_to_write remains the same)
        rows_to_write.append([
            tx.get('id', f'ROW_{i+1}'), tx.get('aasmState', 'N/A'), tx.get('date', 'N/A'),
            get_nested(tx, ['amounts', 'gbpCents']), get_nested(tx, ['amounts', 'referenceCurrency']), get_nested(tx, ['amounts', 'wei']),
            token_op.get('__typename', 'N/A'), get_nested(token_op, ['sender', 'slug'], 'N/A'),
            get_nested(token_op, ['receiver', 'slug'], 'N/A'), get_nested(token_op, ['actualReceiver', 'slug'], 'N/A'),
            ", ".join([c['slug'] for c in get_nested(token_op, ['senderSide', 'anyCards'], [])]),
            ", ".join([c['slug'] for c in get_nested(token_op, ['receiverSide', 'anyCards'], [])]),
            ", ".join([c['slug'] for c in get_nested(token_op, ['auction', 'anyCards'], [])])
        ])

    data_to_overwrite = [standardized_headers] + rows_to_write

    if data_to_overwrite:
        num_rows = len(data_to_overwrite)
        num_cols = len(standardized_headers)
        range_label = f'A1:{gspread.utils.rowcol_to_a1(num_rows, num_cols)}'

        wks_std.update(range_name=range_label, values=data_to_overwrite, value_input_option='RAW')

        print(f"Successfully OVERWROTE {len(rows_to_write)} rows of STANDARDIZED RAW data to '{standardized_worksheet_name}'.")
    else:
        wks_std.clear()
        wks_std.append_row(standardized_headers)
        print("No standardized raw data to dump to Google Sheet. Headers were refreshed.")


def write_to_google_sheet(data_rows, sheet_name, worksheet_name, headers):
    # ... (Code to open spreadsheet and get/create worksheet remains the same) ...
    try:
        sh = gc.open(sheet_name)
    except gspread.SpreadsheetNotFound:
        print(f"Spreadsheet '{sheet_name}' not found. Please create it manually.")
        return

    try:
        wks = sh.worksheet(worksheet_name)
    except gspread.WorksheetNotFound:
        wks = sh.add_worksheet(title=worksheet_name, rows="100", cols=str(len(headers)))
        wks.append_row(headers)

    else: # If sheet exists, clear it for overwrite
        # NOTE: Clearing here is redundant since update() overwrites, but is kept
        # as a safety measure for small ranges if update fails.
        pass

    if data_rows:
        data_to_overwrite = [headers] + data_rows
        num_rows = len(data_to_overwrite)
        num_cols = len(headers)
        range_label = f'A1:{gspread.utils.rowcol_to_a1(num_rows, num_cols)}'

        # FIX: Use named arguments to resolve DeprecationWarning
        wks.update(range_name=range_label, values=data_to_overwrite, value_input_option='RAW')

        print(f"Successfully OVERWROTE {len(data_rows)} rows (including headers) to '{worksheet_name}' in '{sheet_name}'.")
    else:
        wks.clear()
        wks.append_row(headers)
        print(f"No data to write to {worksheet_name}. Headers were refreshed.")

# -------------------------------------------------------------
# --- END WRITING FUNCTIONS ---
# -------------------------------------------------------------

In [58]:
# Cell 9.5: BATCH WRITING FUNCTION (FINAL SYNTAX FIX USING PARAMS)

def write_in_batch(google_sheet_name, all_requests_map):
    """
    Performs all necessary Google Sheet updates in a single batch call.

    Args:
        google_sheet_name (str): Name of the target Google Spreadsheet.
        all_requests_map (dict): Keys are worksheet names (str).
                                 Values are a tuple: (list of headers, list of data rows).
    """
    import gspread.utils

    try:
        sh = gc.open(google_sheet_name)
    except gspread.SpreadsheetNotFound:
        print(f"Error: Spreadsheet '{google_sheet_name}' not found. Cannot perform batch update.")
        return

    requests_body = [] # This will hold the body structure for the API call

    for worksheet_name, (headers, data_rows) in all_requests_map.items():
        # 1. Get or Create the Worksheet
        try:
            wks = sh.worksheet(worksheet_name)
        except gspread.WorksheetNotFound:
            wks = sh.add_worksheet(title=worksheet_name, rows="100", cols=str(len(headers)))
            print(f"-> Created new sheet: {worksheet_name}")

        # 2. Prepare data for overwrite (Headers + Data)
        if data_rows:
            data_to_overwrite = [headers] + data_rows
            num_rows = len(data_to_overwrite)
            num_cols = len(headers)
            # Range must be prefixed with the sheet name!
            range_label = f'{worksheet_name}!A1:{gspread.utils.rowcol_to_a1(num_rows, num_cols)}'

            # Append the required keys for the API call structure
            requests_body.append({
                'range': range_label,
                'values': data_to_overwrite
            })
            print(f"-> Prepared {len(data_rows)} rows for '{worksheet_name}'.")
        else:
            # If no data, clear the sheet and write headers (still one write)
            wks.clear()
            wks.append_row(headers)
            print(f"-> No data for '{worksheet_name}'. Refreshed headers.")

    # 3. Execute Batch Update (consolidated API call)
    if requests_body:
        try:
            # FIX: Use sh.client.request to access the underlying API call directly,
            # passing the value_input_option in the body parameter map (v4 sheets API standard)

            # The URL to batch update sheet values is:
            # /v4/spreadsheets/{spreadsheetId}/values:batchUpdate

            # Prepare the full JSON body payload
            payload = {
                'valueInputOption': 'RAW',
                'data': requests_body
            }

            sh.client.request(
                'post',
                f'https://sheets.googleapis.com/v4/spreadsheets/{sh.id}/values:batchUpdate',
                json=payload
            )

            print("\n✅ Successfully executed BATCH UPDATE for all sheets.")
        except Exception as e:
            # If the raw request fails, try the internal gspread method just in case
            print(f"\n❌ ERROR during direct batch update call: {e}")

            try:
                 sh.values_batch_update(
                    requests_body,
                    value_input_option='RAW'
                )
                 print("\n✅ Successfully executed fallback BATCH UPDATE for all sheets.")
            except Exception as e:
                print(f"\n❌ ERROR: Fallback batch update also failed: {e}")

In [59]:
# CELL 10 PREREQUISITE - GOOGLE SHEETS API
!pip install gspread
# REMOVE: from google.colab import auth
# REMOVE: from google.auth import default
import gspread

# Authenticate with your user account
auth.authenticate_user()

# Get credentials and authorize gspread
creds, _ = default()
gc = gspread.authorize(creds)

print("Google Sheets API access granted successfully.")

Google Sheets API access granted successfully.


In [60]:
# Cell 11: API FETCH LOOP (RAW DATA RETRIEVAL - NO FILTERING)

# Define a list to hold the transactions
transactions = []

# Define the variables for the query
variables = {
   "cursor": None,
}

fetch_counter = 0
fetch_start_time = time.time()

has_next_page = True # Initialize loop condition

# Loop through the pages of results
while has_next_page:
    # Execute the query. 'account_entries' now holds the 'accountEntries' object directly.
    account_entries = execute_query(query, variables, api_key, jwt)

    if account_entries is None:
        print("Stopping fetch due to API error or no data.")
        break

    current_nodes = account_entries["nodes"]
    page_info = account_entries["pageInfo"]

    # Add ALL new transactions to the global list (NO DEDUPLICATION HERE)
    transactions += current_nodes

    variables["cursor"] = page_info["endCursor"]
    has_next_page = page_info["hasNextPage"]
    fetch_counter += len(current_nodes)

# Get the current time after executing the query
fetch_end_time = time.time()
fetch_elapsed_time = math.ceil(fetch_end_time - fetch_start_time)
print(f"\n{fetch_counter} accountEntries records fetched from Sorare API. Query took {fetch_elapsed_time} seconds")
print(f"Total raw transactions retrieved: {len(transactions)} (Full API Dump)")

# The 'transactions' list now contains ALL raw entries.

Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 2

In [61]:
# Cell 12: DATA TRANSFORMATION AND EXPORT LOGIC (Indentation Corrected)

# Define new anomaly sheet name (Moved from last cell for function usage)
ANOMALIES_WORKSHEET_NAME = "Anomalies_Excluded_Data"


# Helper function to safely get a nested value (KEEP THIS)
def get_nested(data, keys, default=None):
   # Safely retrieves a nested value from a dictionary
   if data is None:
       return default
   value = data
   for key in keys:
       if isinstance(value, dict) and key in value:
           value = value[key]
       elif isinstance(value, list) and isinstance(key, int) and key < len(value):
           value = value[key]
       else:
           return default
   return value


# Function to convert Wei (as a string) to ETH (as a float) (KEEP THIS)
def wei_to_eth(wei_str):
   """Converts a wei string value to a floating point ETH value."""
   try:
       wei = int(wei_str)
       eth = wei / 10**18
       return round(eth, 8)
   except (ValueError, TypeError):
       return 0.0


# --- NEW FILTERING FUNCTION ---
def filter_transactions(transactions_list):
    """Filters transactions into a main list (GBP/WEI) and an anomalies list (other currencies)."""
    main_list = []
    anomaly_list = []

    for tx in transactions_list:
        reference_currency = get_nested(tx, ['amounts', 'referenceCurrency'])

        if reference_currency in ["GBP", "WEI"]:
            main_list.append(tx)
        else:
            anomaly_list.append(tx)

    return main_list, anomaly_list


# --- UPDATED MAIN TRANSFORMATION FUNCTION (WITH REWARD_ID) ---
def transform_transaction_data(transactions, user_slug, headers):
    transformed_data = []
    current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    for tx in transactions:
        gbp_cents = get_nested(tx, ['amounts', 'gbpCents'], 0)
        aasm_state = tx.get('aasmState')
        account_entry_id = tx.get('id', '') # <--- EXTRACT ID HERE

        token_op = tx.get('tokenOperation')

        # 🔥 FIX: Explicitly use get_nested to retrieve the TokenOperation ID.
        # This handles nested structure variations more reliably.
        token_operation_id = get_nested(token_op, ['id'], '')

        if token_op is None or aasm_state != 'CONFIRMED':
            continue


        op_type = token_op.get('__typename')

        # Initialize variables
        tx_type = "OTHER"
        main_card_slug = ""
        cards_exchanged_str = ""
        sender = 'N/A'
        receiver = 'N/A'
        reward_id = ""  # <-- Initialize reward_id

        is_classified = False

        # --- SCENARIO 1: TokenOffer (Sale, Offer Buy, Part Exchange) ---
        if op_type == 'TokenOffer':

            sender_slug = get_nested(token_op, ['sender', 'slug'], 'N/A')
            receiver_slug = get_nested(token_op, ['actualReceiver', 'slug'])

            cards_on_sender_side = [c['slug'] for c in get_nested(token_op, ['senderSide', 'anyCards'], [])]
            cards_on_receiver_side = [c['slug'] for c in get_nested(token_op, ['receiverSide', 'anyCards'], [])]


            # SALE LOGIC (Type 1)
            if gbp_cents > 0 and sender_slug == user_slug:
                tx_type = "SALE"
                main_card_slug = ", ".join(cards_on_sender_side)
                cards_exchanged_str = ", ".join(cards_on_receiver_side)
                sender = sender_slug
                receiver = receiver_slug
                is_classified = True

            # BUY LOGIC 1 & 2 (User is Payer: gbpCents < 0)
            elif gbp_cents < 0:
                # Determine user role and corresponding card sides
                if sender_slug == user_slug:
                    cards_user_sent = cards_on_sender_side
                    cards_user_received = cards_on_receiver_side
                    seller_slug = receiver_slug
                    buyer_slug = sender_slug
                elif receiver_slug == user_slug:
                    cards_user_sent = cards_on_receiver_side
                    cards_user_received = cards_on_sender_side
                    seller_slug = sender_slug
                    buyer_slug = receiver_slug
                else:
                    continue

                is_part_exchange = len(cards_user_sent) > 0
                tx_type = "BUY (Part Exchange)" if is_part_exchange else ("BUY (Offer Sent)" if sender_slug == user_slug else "BUY (Offer)")

                main_card_slug = ", ".join(cards_user_received)
                cards_exchanged_str = ", ".join(cards_user_sent)
                sender = seller_slug
                receiver = buyer_slug
                is_classified = True

        # --- SCENARIO 2: TokenBid (Auction Buy) ---
        elif op_type == 'TokenBid' and gbp_cents < 0:
            tx_type = "BUY (Auction)"
            main_card_slug = ", ".join([c['slug'] for c in get_nested(token_op, ['auction', 'anyCards'], [])])
            sender = 'Sorare'
            receiver = user_slug
            is_classified = True

        # --- SCENARIO 3: TokenPrimaryOffer (Instant Buy) ---
        elif op_type == 'TokenPrimaryOffer' and gbp_cents < 0:
            tx_type = "BUY (Instant)"
            main_card_slug = ", ".join([c['slug'] for c in get_nested(token_op, ['anyCards'], [])])
            sender = 'Sorare'
            receiver = get_nested(token_op, ['buyer', 'slug'], user_slug)
            is_classified = True

        # --- SCENARIO 4: MONETARY REWARDS (New) ---
        elif op_type == 'TokenMonetaryReward' and gbp_cents > 0:
            tx_type = "REWARD (Monetary)"
            main_card_slug = ""
            sender = 'Sorare'
            receiver = user_slug
            reward_id = get_nested(token_op, ['rewardId'], 'N/A') # <-- Mapped from rewardId
            is_classified = True

        elif op_type == 'So5Reward' and gbp_cents > 0:
            tx_type = "REWARD (Legacy)"

            # Extract potential card info (as before)
            reward_cards = get_nested(token_op, ['rewardCards'], [])
            card_slugs = []
            for item in reward_cards:
                if 'anyCard' in item and 'slug' in item['anyCard']:
                     card_slugs.append(item['anyCard']['slug'])

            main_card_slug = ", ".join(card_slugs)
            sender = 'Sorare'
            receiver = user_slug
            reward_id = get_nested(token_op, ['slug'], 'N/A') # <-- Mapped from slug
            is_classified = True


        if not is_classified:
            continue

        # ----------------------------------------------------------------------
        # Data Mapping and Currency Formatting
        # ----------------------------------------------------------------------

        card_url = f"https://sorare.com/cards/{main_card_slug.split(',')[0].strip()}" if main_card_slug and main_card_slug != "" else ""

        fiat_value_gbp = gbp_cents / 100.0
        reference_currency = get_nested(tx, ['amounts', 'referenceCurrency'], 'N/A')
        sell_price_wei_str = get_nested(tx, ['amounts', 'wei'])
        crypto_value_eth = wei_to_eth(sell_price_wei_str)

        # Currency Overrides
        if reference_currency == "WEI":
            fiat_value_gbp = "-"
        elif reference_currency == "GBP":
            crypto_value_eth = "-"

        # Date/Time
        full_date_time = tx.get('date', 'N/A')
        if 'T' in full_date_time:
            transaction_date, transaction_time_z = full_date_time.split('T')
            transaction_time = transaction_time_z.rstrip('Z')
        else:
            transaction_date = full_date_time
            transaction_time = 'N/A'

        # Map to Dictionary
        row_dict = {
            "transaction_type": tx_type,
            "card_slug": main_card_slug,
            "card_url": card_url,
            "sender_slug": sender,
            "actual_receiver_slug": receiver,
            "token_op_type": op_type,
            "reward_id": reward_id, # <-- ADDED HERE
            "fiat_value_gbp": fiat_value_gbp,
            "crypto_value_eth": crypto_value_eth,
            "reference_currency": reference_currency,
            "cards_exchanged_slugs": cards_exchanged_str,
            "transaction_date": transaction_date,
            "transaction_time": transaction_time,
            "status": aasm_state,
            "pl_value_gbp": 0.0,
            "current_time": current_time,
            "account_entry_id": account_entry_id,
            "token_operation_id": token_operation_id # <--- MAPPED HERE
        }

        row_list = [row_dict.get(h, '') for h in headers]
        transformed_data.append(row_list)

    return transformed_data


# --- UPDATED ANOMALIES WRITER FUNCTION (Cleaned Syntax and returns data) ---
def prepare_anomaly_data(anomaly_list, user_slug, headers):
    """
    Transforms anomaly data and prepares it for the batch update.
    Returns: Tuple of (headers, data_rows)
    """
    # 1. Use the standard transformation logic
    rows_to_write = transform_transaction_data(anomaly_list, user_slug, headers)

    # 2. Add an ANOMALY flag to the transaction_type column for clarity
    TX_TYPE_INDEX = headers.index('transaction_type')
    for row in rows_to_write:
        if row and len(row) > TX_TYPE_INDEX:
            row[TX_TYPE_INDEX] = "ANOMALY: " + str(row[TX_TYPE_INDEX])

    if rows_to_write:
        print(f"Prepared {len(rows_to_write)} ANOMALOUS records for batch write.")
    else:
        print("No anomalous records found to prepare.")

    return (headers, rows_to_write) # Return the data structure needed for batch write


# --- UPDATED FUNCTION TO SPLIT DATA BY TRANSACTION TYPE (Added Rewards) ---
def split_transactions_by_type(transformed_data):
    """
    Splits the final transformed list into Sales, Buys, and Rewards.
    """
    sales = []
    buys = []
    rewards = [] # <-- New list for rewards

    buy_types = [
        "BUY (Auction)", "BUY (Offer)", "BUY (Part Exchange)",
        "BUY (Adjusted)", "BUY (Offer Sent)", "BUY (Instant)"
    ]
    reward_types = ["REWARD (Monetary)", "REWARD (Legacy)"]

    for row in transformed_data:
        if not row:
            continue

        transaction_type = row[0].replace("ANOMALY: ", "")

        if transaction_type == "SALE":
            sales.append(row)
        elif transaction_type in buy_types:
            buys.append(row)
        elif transaction_type in reward_types: # <-- New reward filter
            rewards.append(row)

    return sales, buys, rewards # <-- Returns three lists

# --- KEEP THESE WRITERS FROM PREVIOUS STEPS (write_raw_data, write_standardized_raw_data, write_to_google_sheet) ---
# NOTE: Ensure you copy all three of these functions from your previous working code into this cell as well.
# They rely on 'get_nested' and 'gc' (Google Sheets client).

In [62]:
# Cell 13: REWARD TRANSFORMATION LOGIC

def transform_rewards_card_data(rewarded_rankings, user_slug, headers):
    transformed_card_rewards = []
    current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    for ranking in rewarded_rankings:
        # Extract the list of rewards associated with this ranking
        so5_rewards_list = ranking.get("so5Rewards", [])

        for so5_reward in so5_rewards_list:

            reward_cards = so5_reward.get("rewardCards", [])

            # --- FILTER: ONLY PROCESS CARD REWARDS ---
            if not reward_cards:
                continue

            # --- Extract Common Ranking/Reward Data ---
            reward_slug = so5_reward.get("slug", "N/A")
            end_date_full = get_nested(so5_reward, ["so5Leaderboard", "endDate"], "N/A")

            transaction_date = end_date_full.split('T')[0] if 'T' in end_date_full else end_date_full
            transaction_time = end_date_full.split('T')[1].rstrip('Z') if 'T' in end_date_full else 'N/A'

            # Get the list of detailed reward types (Monetary, Card, etc.)
            detailed_rewards = so5_reward.get("rewards", [])

            # --- Process Each Card within the Reward ---
            for card_entry in reward_cards:
                main_card_slug = get_nested(card_entry, ["anyCard", "slug"], "")
                if not main_card_slug:
                    continue # Should not happen if filtered correctly

                # Find the corresponding card reward ID and status (optional)
                reward_id = "N/A"
                card_status = "CONFIRMED"

                for reward_detail in detailed_rewards:
                    if reward_detail.get("__typename") == "AnyCardReward":
                        reward_id = reward_detail.get("id", "N/A")
                        card_status = reward_detail.get("aasmState", "CONFIRMED")
                        break

                # --- Map to AccountTransactions Headers ---
                card_url = f"https://sorare.com/cards/{main_card_slug}"

                row_dict = {
                    "transaction_type": "REWARD (Card)",
                    "card_slug": main_card_slug,
                    "card_url": card_url,
                    "sender_slug": "Sorare",
                    "actual_receiver_slug": user_slug,
                    "token_op_type": "TokenReward",
                    "reward_id": reward_id,
                    "reward_slug": reward_slug, # <-- New temporary field to map
                    "fiat_value_gbp": "-",
                    "crypto_value_eth": "-",
                    "reference_currency": "-",
                    "cards_exchanged_slugs": "",
                    "transaction_date": transaction_date,
                    "transaction_time": transaction_time,
                    "status": card_status,
                    "pl_value_gbp": "",
                    "current_time": current_time,
                    "account_entry_id": f"RewardEntry:{reward_id}",
                    "token_operation_id": reward_id # <--- USING REWARD ID
                }

                # Map the dictionary to a list in the correct header order
                row_list = [row_dict.get(h, '') for h in headers]
                transformed_card_rewards.append(row_list)

    return transformed_card_rewards

In [63]:
# Cell 14: CRAFT TRANSFORMATION LOGIC (Card URL Set to Blank)

def transform_craft_data(craft_transactions, user_slug, headers):
    transformed_craft_rewards = []
    current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    for craft_tx in craft_transactions:
        # 1. FILTER: Only process "Craft" transactions
        if craft_tx.get("description") != "Craft":
            continue

        # 2. EXTRACT CARD SLUG (Card:GUID) from the ID field
        # Format is: "CardShardsHistoryTransaction:Card:GUID:DATE_TIME:"
        full_id = craft_tx.get("id", "")

        main_card_slug = "N/A"

        if full_id:
            try:
                # Step A: Split by ':'
                parts = full_id.split(':')

                # Step B: Ensure the necessary parts exist (parts[2] should be the GUID)
                if len(parts) >= 3 and parts[2]:
                    # Check for "Card" prefix and validate GUID length
                    if parts[1] == "Card" and len(parts[2]) == 36:
                        raw_guid = parts[2]
                        main_card_slug = "Card:" + raw_guid  # Final format: Card:GUID
                    else:
                        main_card_slug = "N/A"

                else:
                  main_card_slug = "N/A"

            except Exception as e:
                print(f"Error extracting card GUID from {full_id}: {e}")
                main_card_slug = "N/A"

        # 3. DATE/TIME
        full_date_time = craft_tx.get('date', 'N/A')
        transaction_date = full_date_time.split('T')[0] if 'T' in full_date_time else full_date_time
        transaction_time = full_date_time.split('T')[1].rstrip('Z') if 'T' in full_date_time else 'N/A'


        # 4. MAP TO ACCOUNTTRANSACTIONS HEADERS
        # FIX: Set card_url to an empty string ("") as requested.
        card_url = ""

        row_dict = {
            "transaction_type": "CRAFT REWARD (Card)",
            "card_slug": main_card_slug,
            "card_url": card_url,
            "sender_slug": "Sorare",
            "actual_receiver_slug": user_slug,
            "token_op_type": "TokenCraft",
            "reward_id": full_id,
            "reward_slug": craft_tx.get("label", ""),
            "fiat_value_gbp": "-",
            "crypto_value_eth": "-",
            "reference_currency": "-",
            "cards_exchanged_slugs": "",
            "transaction_date": transaction_date,
            "transaction_time": transaction_time,
            "status": "Crafted",
            "pl_value_gbp": "",
            "current_time": current_time,
            "account_entry_id": craft_tx.get("id", ""),
            "token_operation_id": craft_tx.get("id", "") # <--- USING CRAFT ID
        }

        # Map the dictionary to a list in the correct header order
        row_list = [row_dict.get(h, '') for h in headers]
        transformed_craft_rewards.append(row_list)

    return transformed_craft_rewards

def write_raw_crafts_dump(craft_transactions, sheet_name, raw_crafts_worksheet_name):
    """Writes the RAW JSON string of ALL crafts history to a dedicated sheet for auditing."""
    import json
    raw_headers = ["ID", "Date", "Description", "Raw JSON Data"]

    try:
        sh = gc.open(sheet_name)
    except gspread.SpreadsheetNotFound:
        print(f"Spreadsheet '{sheet_name}' not found. Skipping raw crafts dump.")
        return

    try:
        wks_raw = sh.worksheet(raw_crafts_worksheet_name)
        wks_raw.clear() # Clear existing content if sheet is found
    except gspread.WorksheetNotFound:
        wks_raw = sh.add_worksheet(title=raw_crafts_worksheet_name, rows="100", cols="4")

    # Prepare data for overwrite
    rows_to_write = []
    for i, tx in enumerate(craft_transactions):
        transaction_id = tx.get('id', f'ROW_{i+1}')
        raw_json_string = json.dumps(tx)

        # Include key auditing fields for easy sorting in the sheet
        rows_to_write.append([
            transaction_id,
            tx.get('date', 'N/A'),
            tx.get('description', 'N/A'),
            raw_json_string
        ])

    data_to_overwrite = [raw_headers] + rows_to_write

    if data_to_overwrite:
        num_rows = len(data_to_overwrite)
        num_cols = len(raw_headers)
        range_label = f'A1:{gspread.utils.rowcol_to_a1(num_rows, num_cols)}'

        wks_raw.update(range_name=range_label, values=data_to_overwrite, value_input_option='RAW')

        print(f"Successfully OVERWROTE {len(rows_to_write)} rows of RAW CRAFTS HISTORY data to '{raw_crafts_worksheet_name}'.")
    else:
        wks_raw.clear()
        wks_raw.append_row(raw_headers)
        print("No raw crafts history data to dump. Headers were refreshed.")

In [64]:
# Cell 15: DIRECT OFFER TRANSFORMATION AND SPLITTING LOGIC (UPDATED)

# Define new sheet name variables for clarity
NOTIONAL_SALES_WORKSHEET_NAME = "Notional_Sales_Auto"
DIRECT_OFFERS_MANUAL_WORKSHEET_NAME = "Direct_Offers_Manual_Audit"

def transform_all_direct_offers(raw_offers, user_slug, headers):
    """
    Transforms ALL raw accepted direct offer data into a single audit list.
    Does NOT perform the final split or fallout classification.
    """
    transformed_data = []
    current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    for offer in raw_offers:

        # --- Initialize Core Variables to avoid NameError ---
        user_sent_cards = []
        user_received_cards = []
        payment_ref_currency = '-'
        final_fiat_value = '-'
        final_crypto_value = '-'
        main_card_slug = ''

        # Get raw side data
        sender_cards = [c['slug'] for c in get_nested(offer, ['senderSide', 'anyCards'], [])]
        receiver_cards = [c['slug'] for c in get_nested(offer, ['receiverSide', 'anyCards'], [])]


        # --- Set Flow based on User's Role ---

        if offer.get('direction_role') == 'RECEIVED':
            # User is the OFFER RECEIVER (Seller). Other party is the SENDER (Buyer).

            # User's flow: Cards sent via receiverSide, Money/Cards received via senderSide.
            user_sent_cards = receiver_cards
            user_received_cards = sender_cards

            # Use the SENDER's amounts as the payment received by the user.
            payment_fiat_cents = get_nested(offer, ['senderSide', 'amounts', 'gbpCents'], 0)
            payment_wei = get_nested(offer, ['senderSide', 'amounts', 'wei'], '0')
            payment_ref_currency = get_nested(offer, ['senderSide', 'amounts', 'referenceCurrency'], '-')

            # Final values for output columns (Sale is positive cash flow)
            final_fiat_value = payment_fiat_cents / 100.0
            final_crypto_value = wei_to_eth(payment_wei)

            main_card_slug = ", ".join(user_sent_cards) or ", ".join(user_received_cards)

        elif offer.get('direction_role') == 'SENT':
            # User is the OFFER SENDER (Buyer or Trader). Other party is the RECEIVER.

            # User's flow: Cards sent via senderSide, Money/Cards received via receiverSide.
            user_sent_cards = sender_cards
            user_received_cards = receiver_cards

           # Payment (if any) is complex/negative and is handled by manual audit anyway.
           # We rely on the AE cross-check. Defaulting cash values to '-' and currency to '-'.
            final_fiat_value = '-'
            final_crypto_value = '-'
            payment_ref_currency = '-'

        else: # Should not happen
            continue

        # Map the offer into a generic audit row
        tx_type = f"OFFER_TRADE ({offer['direction_role']})"

        full_date_time = offer.get('acceptedAt', offer.get('startDate', 'N/A'))
        transaction_date = full_date_time.split('T')[0] if 'T' in full_date_time else full_date_time
        transaction_time = full_date_time.split('T')[1].rstrip('Z') if 'T' in full_date_time else 'N/A'

        sender_final = get_nested(offer, ['sender', 'slug'], 'N/A')
        receiver_final = get_nested(offer, ['receiver', 'slug'], 'N/A')


        # Determine Cards Exchanged String based on the user's perspective
        cards_exchanged_str = f"SENT: {', '.join(user_sent_cards)} | RECEIVED: {', '.join(user_received_cards)}"

        row_dict = {
            "transaction_type": tx_type,
            "card_slug": main_card_slug,
            "card_url": "", # Keep blank as per previous requirement
            "sender_slug": sender_final,
            "actual_receiver_slug": receiver_final,
            "token_op_type": "TokenOffer",
            "reward_id": offer.get('id', ''),
            "reward_slug": offer['direction_role'],
            # The signs on fiat_output/crypto_output now correctly represent the cash flow (Sale = positive)
            # 🔥 CRITICAL: Map payment details directly from senderSide (the buyer) 🔥
            "fiat_value_gbp": final_fiat_value,
            "crypto_value_eth": final_crypto_value,
            "reference_currency": payment_ref_currency, # Capture the ACTUAL CURRENCY (EUR, WEI, GBP, etc.)
            "cards_exchanged_slugs": cards_exchanged_str, # Use corrected string
            "transaction_date": transaction_date,
            "transaction_time": transaction_time,
            "status": offer['status'].upper(),
            "pl_value_gbp": 0.0,
            "current_time": current_time,
            # Placeholder ID for initial cross-check
            "account_entry_id": f"OfferPreCheck:{offer.get('id', '')}",
            "token_operation_id": offer.get('id', '')
        }
        transformed_data.append([row_dict.get(h, '') for h in headers])

    return transformed_data


def notional_sale_mapper(offer, user_slug, fiat_value_gbp, crypto_value_eth, reference_currency, card_slugs, headers, current_time):
    """Maps a Notional Sale to the standard transaction format."""

    # Currency Overrides (as per your request)
    if reference_currency not in ["GBP", "WEI"]:
         reference_currency = "GBP_FOREIGN_NOTIONAL" # Use a specific flag

    # Format fiat/crypto values for output
    fiat_output = round(fiat_value_gbp, 2) if fiat_value_gbp > 0 and reference_currency == "GBP" else "-"
    crypto_output = round(crypto_value_eth, 8) if crypto_value_eth > 0 and reference_currency == "WEI" else "-"
    if reference_currency == "GBP_FOREIGN_NOTIONAL":
        fiat_output = round(fiat_value_gbp, 2) # Assume fiat value is the GBP value for simplicity


    full_date_time = offer.get('acceptedAt', offer.get('startDate', 'N/A'))
    transaction_date = full_date_time.split('T')[0] if 'T' in full_date_time else full_date_time
    transaction_time = full_date_time.split('T')[1].rstrip('Z') if 'T' in full_date_time else 'N/A'

    row_dict = {
        "transaction_type": "NOTIONAL_SALE",
        "card_slug": card_slugs[0], # Only one card for this type
        "card_url": f"https://sorare.com/cards/{card_slugs[0]}",
        "sender_slug": user_slug, # User is the seller
        "actual_receiver_slug": get_nested(offer, ['sender', 'slug'], 'N/A'),
        "token_op_type": "TokenOffer",
        "reward_id": offer.get('id', ''),
        "reward_slug": "SALE_VIA_OFFER",
        "fiat_value_gbp": fiat_output,
        "crypto_value_eth": crypto_output,
        "reference_currency": reference_currency,
        "cards_exchanged_slugs": "",
        "transaction_date": transaction_date,
        "transaction_time": transaction_time,
        "status": "PROCESSED_NOTIONAL",
        "pl_value_gbp": 0.0,
        "current_time": current_time,
        "account_entry_id": "NOTIONAL_SALE", # Explicitly mark as NOTIONAL for de-dupe
        "token_operation_id": offer.get('id', '')
    }
    return [row_dict.get(h, '') for h in headers]


def direct_offer_audit_mapper(offer, user_slug, headers, current_time):
    """Maps a manual/audit direct offer to the standard transaction format."""
    # ... (Logic from previous Cell 13.5 remains largely the same for AUDIT rows) ...
    # Simplified mapping for audit: just grab the overall offer details
    tx_type = f"OFFER_TRADE ({offer['direction_role']})"

    full_date_time = offer.get('acceptedAt', offer.get('startDate', 'N/A'))
    transaction_date = full_date_time.split('T')[0] if 'T' in full_date_time else full_date_time
    transaction_time = full_date_time.split('T')[1].rstrip('Z') if 'T' in full_date_time else 'N/A'

    sender = get_nested(offer, ['sender', 'slug'], 'N/A')
    receiver = get_nested(offer, ['receiver', 'slug'], 'N/A')

    sender_cards = [c['slug'] for c in get_nested(offer, ['senderSide', 'anyCards'], [])]
    receiver_cards = [c['slug'] for c in get_nested(offer, ['receiverSide', 'anyCards'], [])]

    # We will let the cross-check apply the FALLOUT status to the account_entry_id later

    row_dict = {
        "transaction_type": tx_type,
        "card_slug": ", ".join(receiver_cards) or ", ".join(sender_cards),
        "card_url": "",
        "sender_slug": sender,
        "actual_receiver_slug": receiver,
        "token_op_type": "TokenOffer",
        "reward_id": offer.get('id', ''),
        "reward_slug": offer['direction_role'],
        "fiat_value_gbp": "-",
        "crypto_value_eth": "-",
        "reference_currency": "-",
        "cards_exchanged_slugs": f"SENT: {', '.join(sender_cards)} | RECEIVED: {', '.join(receiver_cards)}",
        "transaction_date": transaction_date,
        "transaction_time": transaction_time,
        "status": offer['status'].upper(), # Use UPPERCASE for clarity
        "pl_value_gbp": 0.0,
        "current_time": current_time,
        "account_entry_id": f"DirectOffer:{offer.get('id', '')}", # Default placeholder ID
        "token_operation_id": offer.get('id', '')
    }
    return [row_dict.get(h, '') for h in headers]

In [65]:
# Cell 16: DIRECT OFFER CROSS-CHECK LOGIC (REVERTED TO DIRECT TOKEN ID LOOKUP)

def cross_check_direct_offers(sales_data, buys_data, monetary_rewards_data, manual_audit_data, headers):
    """
    Checks each Offer against main transaction data using the token_operation_id directly.

    If a match is found: sets account_entry_id = matching transaction's account_entry_id.
    If no match is found: sets account_entry_id = 'FALLOUT'.
    """
    import pandas as pd

    # 1. Map headers to column indices
    header_indices = {header: i for i, header in enumerate(headers)}
    AE_ID_COL = header_indices.get("account_entry_id")
    TO_ID_COL = header_indices.get("token_operation_id")

    if AE_ID_COL is None or TO_ID_COL is None:
        print("Error: Required headers (account_entry_id, token_operation_id) not found.")
        return manual_audit_data

    # 2. Consolidate and prepare Financial Data
    all_financial_data_rows = sales_data + buys_data + monetary_rewards_data
    financial_df = pd.DataFrame(all_financial_data_rows, columns=headers)

    # Create lookup map: {token_operation_id: account_entry_id}
    # Use drop_duplicates(keep='last') to handle cases where a single token operation ID
    # might map to multiple account entries (common with fees/splits), keeping the most relevant one.
    financial_lookup = financial_df.drop_duplicates(
        subset=['token_operation_id'], keep='last'
    ).set_index('token_operation_id')['account_entry_id'].to_dict()


    # 3. Process Direct Offers for Lookup
    updated_direct_offers = []

    for row in manual_audit_data:

        offer_token_op_id = row[TO_ID_COL]

        # Perform the DIRECT lookup (no normalization needed)
        matched_ae_id = financial_lookup.get(offer_token_op_id)

        if matched_ae_id:
            # Match found: The offer is covered by a confirmed account entry
            row[AE_ID_COL] = matched_ae_id
        else:
            # No match found: This is a genuine fallout/unrecorded trade
            row[AE_ID_COL] = 'FALLOUT'

        updated_direct_offers.append(row)

    print(f"Manual Audit Offer Cross-Check complete. Updated {len(updated_direct_offers)} records.")

    return updated_direct_offers

In [66]:
# Cell 17: SPLIT FALLOUT DIRECT OFFERS (Updated with URL Generation)

def split_fallout_direct_offers(all_offers_list, user_slug, headers):
    """
    Splits the full list of cross-checked offers (where fallout has been applied)
    into Notional Sales (FALLOUT + Sale Pattern) and Manual Audit (FALLOUT + Others / PROCESSED).
    """
    notional_sales_data = []
    final_manual_audit_data = [] # Retains processed, fallout, and general audit entries

    # Map headers to column indices
    header_indices = {header: i for i, header in enumerate(headers)}
    AE_ID_COL = header_indices.get("account_entry_id")
    TO_ID_COL = header_indices.get("token_operation_id")
    TX_TYPE_COL = header_indices.get("transaction_type")
    CARDS_EXCHANGED_COL = header_indices.get("cards_exchanged_slugs")
    CARD_SLUG_COL = header_indices.get("card_slug") # NEW INDEX
    CARD_URL_COL = header_indices.get("card_url") # NEW INDEX

    # Check for required headers
    if AE_ID_COL is None or TO_ID_COL is None or TX_TYPE_COL is None or CARD_SLUG_COL is None or CARD_URL_COL is None:
        print("Error: Required headers not found for splitting. Returning all to Manual Audit.")
        return [], all_offers_list

    # Get indexes for currency/status updates
    fiat_col = header_indices.get("fiat_value_gbp")
    crypto_col = header_indices.get("crypto_value_eth")
    ref_col = header_indices.get("reference_currency")

    for row in all_offers_list:

        # 1. Check if it's a FALLOUT
        is_fallout = (row[AE_ID_COL] == 'FALLOUT')

        # 2. Check for Notional Sale Pattern (Only execute if it's a FALLOUT)

        cards_exchanged_string = row[CARDS_EXCHANGED_COL]
        is_notional_sale_pattern = False # Initialize to False

        if is_fallout:

            try:
                sent_part, received_part = cards_exchanged_string.split(' | RECEIVED: ')

                cards_sent_slugs_str = sent_part.replace('SENT: ', '').strip()
                cards_received_slugs_str = received_part.strip()

                # A. Check for exactly ONE card SENT by the user (sold card)
                user_sent_one_card = (cards_sent_slugs_str != '' and ',' not in cards_sent_slugs_str)

                # B. Check for ZERO cards RECEIVED by the user
                user_received_zero_cards = (cards_received_slugs_str == '')

                is_notional_sale_pattern = user_sent_one_card and user_received_zero_cards

            except ValueError:
                pass


        if is_notional_sale_pattern:
            # --- Map to Notional Sale ---

            # 1. Generate Card URL (Remains the same)
            card_slug = row[CARD_SLUG_COL]
            if card_slug and ' ' not in card_slug:
                 row[CARD_URL_COL] = f"https://sorare.com/cards/{card_slug}"

            # Get the current values and reference currency from the row
            original_ref_currency = row[ref_col]
            current_fiat_value = row[fiat_col]
            # current_crypto_value is already in ETH from Cell 15
            current_crypto_value = row[crypto_col]

            # --- 🔥 2. Apply Exclusive Currency Rules (The Final Fix) 🔥

            if original_ref_currency == "WEI":
                # Rule (1): WEI
                row[ref_col] = "WEI"
                row[fiat_col] = "-"                   # Dash out Fiat
                # row[crypto_col] remains as is (populated with ETH)

            elif original_ref_currency == "GBP":
                # Rule (2): GBP
                row[ref_col] = "GBP"
                row[crypto_col] = "-"                 # Dash out Crypto
                # row[fiat_col] remains as is (populated with GBP amount)

            else:
                # Rule (3): GBP_FOREIGN (EUR, USD, etc.)
                row[ref_col] = "GBP_FOREIGN"
                row[crypto_col] = "-"                 # Dash out Crypto
                # row[fiat_col] remains as is (populated with the GBP Cents equivalent)


            # 3. Finalize Status and IDs
            row[TX_TYPE_COL] = "NOTIONAL_SALE"
            row[header_indices.get("status")] = "PROCESSED_NOTIONAL"
            row[AE_ID_COL] = "NOTIONAL_SALE_MAPPED"

            # Append to sales
            notional_sales_data.append(row)
        else:
            # --- Map to Manual Audit ---
            final_manual_audit_data.append(row)

    print(f"Split complete: {len(notional_sales_data)} Notional Sales isolated. {len(final_manual_audit_data)} remaining for Manual Audit.")

    return notional_sales_data, final_manual_audit_data

In [67]:
# SCRIPT 1: Cell 17.1 (Fetch and Initial Processing)

if 'transactions' in locals() and transactions:
    print("--- Starting API Fetches and Initial Processing ---")

    # 1. Fetch auxiliary data
    rewards_rankings = fetch_all_rewards(rewards_query, user, api_key, jwt)
    craft_transactions = fetch_all_crafts(crafts_query, api_key, jwt)
    direct_offers = fetch_all_direct_offers(direct_offers_query, user, api_key, jwt)

    # 2. Filter and Process Main Data
    main_transactions, anomaly_transactions = filter_transactions(transactions)
    data_to_write = transform_transaction_data(main_transactions, user, headers)
    sales_data, buys_data, monetary_rewards_data = split_transactions_by_type(data_to_write)

    # 3. Process Auxiliary Data
    card_rewards_data = transform_rewards_card_data(rewards_rankings, user, headers)
    crafted_data = transform_craft_data(craft_transactions, user, headers)

    # 4. Process Direct Offers
    all_direct_offers_audit_data = transform_all_direct_offers(direct_offers, user, headers)
    all_direct_offers_audit_data = cross_check_direct_offers(
        sales_data, buys_data, monetary_rewards_data, all_direct_offers_audit_data, headers
    )
    notional_sales_data, direct_offers_manual_audit_data = split_fallout_direct_offers(
        all_direct_offers_audit_data, user, headers
    )
    print("--- Initial Data Processing Complete ---")
else:
    print("Error: 'transactions' list is empty or not defined. Rerun API fetch cells.")

--- Starting API Fetches and Initial Processing ---
Fetching rewards data...
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Status Code: 200
Statu

In [68]:
# SCRIPT 1: Cell 17.2 (Processing and Audit Trail)

if 'sales_data' in locals():
    print("--- Starting Audit and Sorting ---")

    # Get the index of the 'token_operation_id', 'status', and 'account_entry_id' columns
    TO_ID_INDEX = headers.index('token_operation_id')
    STATUS_INDEX = headers.index('status')
    DATE_INDEX = headers.index('transaction_date')
    AE_ID_INDEX = headers.index('account_entry_id')

    # 5. Flag resolved trades in Direct_Offers_Manual_Audit

    # A. Load the list of manually resolved Token Operation IDs
    try:
        import pandas as pd
        wks_manual = gc.open(GOOGLE_SHEET_NAME).worksheet(MANUAL_SALES_WORKSHEET_NAME)
        manual_sales_df = pd.DataFrame(wks_manual.get_all_records())
        resolved_token_ids = set(manual_sales_df['token_operation_id'].astype(str).tolist())
        print(f"Loaded {len(resolved_token_ids)} manually resolved trades from '{MANUAL_SALES_WORKSHEET_NAME}'.")
    except Exception as e:
        resolved_token_ids = set()
        print(f"Error loading manual sales data for flagging: {e}. Skipping flag.")


    # B. Update the status in the manual audit list
    resolved_count = 0
    for row in direct_offers_manual_audit_data:
        token_op_id = str(row[TO_ID_INDEX])

        is_fallout = row[AE_ID_INDEX] == 'FALLOUT'

        if is_fallout and token_op_id in resolved_token_ids:
             row[STATUS_INDEX] = "RESOLVED_TO_MANUAL_SALE"
             row[AE_ID_INDEX] = "MANUAL_SALE_RESOLVED"
             resolved_count += 1

    print(f"Flagged {resolved_count} records in '{DIRECT_OFFERS_MANUAL_WORKSHEET_NAME}' as resolved.")


    # 6. SORTING LOGIC
    notional_sales_data.sort(key=lambda x: x[DATE_INDEX], reverse=True)
    direct_offers_manual_audit_data.sort(key=lambda x: x[DATE_INDEX], reverse=True)
    print("Sorted Notional Sales and Manual Audit by date (newest first).")

else:
    print("Run Cell 17.1 first to generate data structures.")

--- Starting Audit and Sorting ---
Loaded 12 manually resolved trades from 'Notional_Sales_Manual'.
Flagged 12 records in 'Direct_Offers_Manual_Audit' as resolved.
Sorted Notional Sales and Manual Audit by date (newest first).


In [69]:
# SCRIPT 1: Cell 18 (Batch Write and Snapshot Archival)

if 'sales_data' in locals():
    print("--- Starting Batch Write and Snapshot ---")

    # 7. FULL BATCH WRITE SETUP
    all_sheets_data = {}

    # 8. CONDITIONAL RAW DUMPS (Logic copied from previous step)
    import json
    # Ensure RUN_RAW_DUMPS is available (defined in a previous cell)
    if 'RUN_RAW_DUMPS' not in locals(): RUN_RAW_DUMPS = False

    if RUN_RAW_DUMPS:
        print("Preparing RAW Dumps...")
        # --- Raw JSON Dump ---
        raw_json_headers = ["ID", "Raw JSON Data"]
        raw_json_data = [[tx.get('id', f'ROW_{i+1}'), json.dumps(tx)] for i, tx in enumerate(transactions)]
        all_sheets_data[RAW_WORKSHEET_NAME] = (raw_json_headers, raw_json_data)

        # --- Standardized Raw Dump (Headers and data must be derived as before) ---
        # ... (Insert full logic for Standardized Raw Dump preparation here) ...

        # --- Raw Crafts Audit Dump (Headers and data must be derived as before) ---
        # ... (Insert full logic for Raw Crafts Audit Dump preparation here) ...

    # 9. Add the main, standard sheets
    all_sheets_data[SALES_WORKSHEET_NAME] = (headers, sales_data)
    all_sheets_data[BUYS_WORKSHEET_NAME] = (headers, buys_data)
    all_sheets_data[REWARDS_WORKSHEET_NAME] = (headers, monetary_rewards_data)
    all_sheets_data[CARD_REWARDS_WORKSHEET_NAME] = (headers, card_rewards_data)
    all_sheets_data[CRAFTED_CARDS_WORKSHEET_NAME] = (headers, crafted_data)
    all_sheets_data[NOTIONAL_SALES_WORKSHEET_NAME] = (headers, notional_sales_data)
    all_sheets_data[DIRECT_OFFERS_MANUAL_WORKSHEET_NAME] = (headers, direct_offers_manual_audit_data)

    # 10. Execute Batch Write
    write_in_batch(GOOGLE_SHEET_NAME, all_sheets_data)

    # 11. Write Anomalies
    write_anomalies(anomaly_transactions, GOOGLE_SHEET_NAME, ANOMALIES_WORKSHEET_NAME, user, headers)

    # 12. ARCHIVAL SNAPSHOT
    try:
        main_sh = gc.open(GOOGLE_SHEET_NAME)
        snapshot_folder_id = get_snapshot_folder_id(SNAPSHOT_FOLDER_NAME)

        if snapshot_folder_id:
            create_and_store_snapshot(
                source_file_id=main_sh.id,
                source_file_name=GOOGLE_SHEET_NAME,
                target_folder_id=snapshot_folder_id
            )
        else:
            print("Snapshot skipped due to folder access error.")

    except Exception as e:
        print(f"CRITICAL: Failed to locate or snapshot the main workbook: {e}")

else:
    print("Run Cell 17.1 first to generate data structures.")

--- Starting Batch Write and Snapshot ---
-> Prepared 2878 rows for 'AccountTransactions_Sales'.
-> Prepared 2586 rows for 'AccountTransactions_Buys'.
-> Prepared 1635 rows for 'AccountTransactions_MonetaryRewards'.
-> Prepared 3202 rows for 'MyRewards_Cards'.
-> Prepared 90 rows for 'MyCrafts_Cards'.
-> Prepared 697 rows for 'Notional_Sales_Auto'.
-> Prepared 272 rows for 'Direct_Offers_Manual_Audit'.

✅ Successfully executed BATCH UPDATE for all sheets.
Successfully OVERWROTE 65 ANOMALOUS records to 'Anomalies_Excluded_Data'.
Drive API service initialized.
✅ Snapshot 'Sorare Transactions Dump_Snapshot_20251027_093624' successfully created and stored in 'SorareReportsSnapshots'.
